# Optimized Speech-to-Text Model with NLP Tasks
This notebook implements a speech-to-text model using Wav2Vec2 and applies NLP tasks such as sentiment analysis, classification, and summarization while optimizing RAM usage.

In [ ]:
!pip install torch torchaudio transformers datasets librosa nltk textblob scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import librosa
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from textblob import TextBlob
from transformers import pipeline

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")

# Convert model to half precision to reduce RAM usage
model.half()
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [ ]:
# Define your file path (REPLACE with actual path)
audio_path = "/content/sample_data/M4 Macbook Air Review： Too Easy!.mp3"

# Load the audio
speech, sr = librosa.load(audio_path, sr=16000)

chunk_duration = 30  # Increase chunk size
total_duration = len(speech) / sr
transcriptions = []

for start in range(0, int(total_duration), chunk_duration):
    end = min(start + chunk_duration, total_duration)
    chunk = speech[int(start * sr): int(end * sr)]

    input_values = processor(chunk, return_tensors="pt", sampling_rate=16000).input_values.half()

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcriptions.append(processor.batch_decode(predicted_ids)[0])

    print(f"Processed {start}-{end} seconds")

# Join all transcriptions
transcription = " ".join(transcriptions)
print("Final Transcription:", transcription)


Processed 0-30 seconds


In [ ]:

sentiment = TextBlob(transcription).sentiment.polarity
sentiment_label = "Positive" if sentiment > 0 else "Negative" if sentiment < 0 else "Neutral"
print(f"Sentiment: {sentiment_label} (Score: {sentiment})")


In [ ]:

categories = {
    "Technology": ["AI", "machine learning", "computer", "software"],
    "Automotive": ["car", "engine", "vehicle", "Tesla"],
    "Sports": ["football", "basketball", "tennis"],
    "Entertainment": ["movie", "music", "show"]
}

def classify_text(text):
    text = text.lower()
    for category, keywords in categories.items():
        if any(keyword in text for keyword in keywords):
            return category
    return "Unknown"

classification = classify_text(transcription)
print(f"Classification: {classification}")


In [ ]:

summarizer = pipeline("summarization")
summary = summarizer(transcription, max_length=50, min_length=10, do_sample=False)[0]['summary_text']
print("Summary:", summary)


In [ ]:

with open("results.txt", "w") as f:
    f.write(f"Transcription:\n{transcription}\n\n")
    f.write(f"Sentiment: {sentiment_label} (Score: {sentiment})\n")
    f.write(f"Classification: {classification}\n")
    f.write(f"Summary:\n{summary}\n")

print("Results saved to results.txt")
